# TP : apprentissage multimodal


Dans ce TP, nous allons utiliser le modèle d'apprentissage, FashionCLIP, pré-entraîné sur des images ainsi que des descriptions en langage naturel. Plus particulièrement, nous allons considérer deux cas d'usage :

*   **Moteur de recherche d'images :** il s'agit de trouver, à partir d'une requête en langage naturel, l'image correspondante.

*   **Classification zero-shot :** il s'agit simplement de construire un classifieur d'images (faire correspondre un label à une image).



## Dataset

Nous allons dans un premier temps télécharger les données. Celles-ci provienennt de [Kaggle](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations).

In [ ]:
%%capture
!pip install gdown
!gdown "1igAuIEW_4h_51BG1o05WS0Q0-Cp17_-t&confirm=t"
!unzip data

### Modèle FashionCLIP

Nous allons également télécharger le modèle pré-entraîné.

In [ ]:
%%capture
!pip install -U fashion-clip

In [ ]:
import sys
#sys.path.append("fashion-clip/")
from fashion_clip.fashion_clip import FashionCLIP
import pandas as pd
import numpy as np
from collections import Counter
from PIL import Image
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression

In [ ]:
%%capture
fclip = FashionCLIP('fashion-clip')

FashionCLIP, à l'instar de CLIP, crée un espace vectoriel partagé pour les images et le texte. Cela permet de nombreuses applications, telles que la recherche (trouver l'image la plus similaire à une requête donnée) ou la classification zero-shot.

Il y a principalement deux composants : un encodeur d'image (pour générer un vecteur à partir d'une image) et un encodeur de texte (pour générer un vecteur à partir d'un texte).










<img src="https://miro.medium.com/v2/resize:fit:1400/0*FLNMtW6jK51fm7Og"  width="400">



Nous allons télécharger les données que nous allons ensuite nettoyer.

In [ ]:
articles = pd.read_csv("data_for_fashion_clip/articles.csv")

# Supprimer les éléments ayant la même description
subset = articles.drop_duplicates("detail_desc").copy()

# Supprimer les images dont la catégrie n'est pas renseignée
subset = subset[~subset["product_group_name"].isin(["Unknown"])]

# Garder seulement les descriptions dont la longueur est inférieure à 40 tokens
subset = subset[subset["detail_desc"].apply(lambda x : 4 < len(str(x).split()) < 40)]

# Supprimer les articles qui ne sont pas suffisamment fréquents dans le jeu de données
most_frequent_product_types = [k for k, v in dict(Counter(subset["product_type_name"].tolist())).items() if v > 10]
subset = subset[subset["product_type_name"].isin(most_frequent_product_types)]

subset.head(3)

In [ ]:
subset.columns

In [ ]:
subset.to_csv("subset_data.csv", index=False)
f"Il y a {len(subset)} éléments dans le dataset"

## Moteur de recherche d'images

Constuire un moteur de recherche qui permet, à partir d'une description en langage naturel, de récupérer l'image correspondante. Mesurer ses performances (précision).

<img src="https://miro.medium.com/v2/resize:fit:1400/1*cnKHgLAumVyuHuK9pkqr7A.gif"  width="800">


In [ ]:
images = ["data_for_fashion_clip/" + str(k) + ".jpg" for k in subset["article_id"].tolist()]
texts = subset["detail_desc"].tolist()

# Créer les représentations vectorielles (embeddings) des images et des descriptions.
image_embeddings = fclip.encode_images(images, batch_size=32)
text_embeddings = fclip.encode_text(texts, batch_size=32)

In [ ]:
print(image_embeddings.shape)
print(text_embeddings.shape)

In [ ]:
def find_closest_image(text_embeddings, image_embeddings,images, text_idx=0, top_k=5):
    embedded_txt = text_embeddings[text_idx]
    # pour chaque image, on calcule cosine similarity
    similarities = []
    for image_embedding in image_embeddings:
        similarities.append(np.dot(embedded_txt, image_embedding) / (np.linalg.norm(embedded_txt) * np.linalg.norm(image_embedding)))
    # on recupère les top_k images les plus proches
    closest_image_idxs = np.argsort(similarities)[::-1][:top_k]
    return closest_image_idxs

In [ ]:
# calcul de l'accuracy du modèle
def compute_accuracy(text_embeddings, image_embeddings, images):
    accuracy = 0
    for idx, text_embedding in enumerate(text_embeddings):
        closest_image_idxs = find_closest_image(text_embeddings, image_embeddings, images, text_idx=idx, top_k=5)
        closest_images = [images[k] for k in closest_image_idxs]
        if images[idx] in closest_images:
            accuracy += 1
    return accuracy / len(text_embeddings)

In [ ]:
accuracy = compute_accuracy(text_embeddings, image_embeddings, images)
print(f"Accuracy: {accuracy}")

# Classification zero-shot

Construite un classsifieur d'images (prédire le label d'une image). Mesurer ses performances.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*No6ZONpQMIcfFaNMOI5oNw.gif"  width="800">



In [ ]:
def find_closest_label(image_embeddings, text_embeddings, labels, image_idx=0, top_k=5):
    embedded_img = image_embeddings[image_idx]
    # pour chaque texte, calcul de la cosine similarity
    similarities = []
    for text_embedding in text_embeddings:
        similarities.append(np.dot(embedded_img, text_embedding) / (np.linalg.norm(embedded_img) * np.linalg.norm(text_embedding)))
    # reucuperation des top_k labels les plus proches
    closest_label_idxs = np.argsort(similarities)[::-1][:top_k]
    return [labels[k] for k in closest_label_idxs]

def compute_label_accuracy(image_embeddings, text_embeddings, labels):
    accuracy = 0
    for idx, image_embedding in enumerate(image_embeddings):
        closest_labels = find_closest_label(image_embeddings, text_embeddings, labels, image_idx=idx, top_k=5)
        if labels[idx] in closest_labels:
            accuracy += 1
    return accuracy / len(image_embeddings)

# accuracy
label_accuracy = compute_label_accuracy(image_embeddings, text_embeddings, texts)
print(f"Label Accuracy: {label_accuracy}")